## Add hexapod motions to the mount plots

Craig Lage  04-Jun-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors, \
        getAltAzOverPeriod, getAzElRotHexDataForExposure
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all'])

In [ ]:
#expId = 2025071800254
#expId = 2025071800445
#expId = 2025071800441
expId = 2025102200082
#expId = 2025102300193
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)

In [ ]:
(mountErrors, mountData) = calculateMountErrors(expRecord, client)
print(mountErrors)
saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Mount_Plot_Hex_{expId}.png"
fig = plotMountErrors(mountData, mountErrors, saveFilename=saveFilename)

In [ ]:
begin = expRecord.timespan.begin
begin.utc.isot

In [ ]:
ttimes = mountData.azimuthData['actualPositionTimestamp']

In [ ]:
ttimes[0]

In [ ]:
Time(ttimes[0], format='unix_tai').utc.isot

In [ ]:
times = mountData.azimuthData.actualPositionTimestamp.values
azs = mountData.azimuthData.actualPosition.values
plt.plot(times, azs, marker='x')
demTimes = mountData.azimuthData.demandPositionTimestamp.values
azDem = mountData.azimuthData.demandPosition.values
plt.plot(demTimes, azDem, marker='o', label='demandPosition')
plt.plot(times, azs, marker='x', label='actualPosition')
plt.xlabel("Time(unix_tai)")
plt.ylabel("Azimuth(degrees)")
plt.legend()
plt.title(f"Plot of azimuth actual position vs actual position timestamp\n{expId}")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Uneven_Timestamps_{expId}.png")

In [ ]:
plt.plot(times, azs, marker='x')

In [ ]:
mountData.camhexData['position0'].plot()
mountData.camhexData['demand0'].plot()

In [ ]:
rotVals = mountData.rotationData['actualPosition'].values
rotRate = (rotVals[-1] - rotVals[0]) / 30.0
print(rotRate)

In [ ]:
import astropy.units as u
from astropy.coordinates import EarthLocation
az = np.median(mountData.azimuthData['actualPosition'].values)
el = np.median(mountData.elevationData['actualPosition'].values)
SIMONYI_LOCATION = EarthLocation.of_site("Rubin:Simonyi")
EARTH_ROTATION = 15.04106858  # degrees/hour

modelRotRate = (
        -EARTH_ROTATION
        * np.cos(SIMONYI_LOCATION.lat.rad)
        * np.cos(az * u.deg)
        / np.cos(el * u.deg)
        / 3600.0
    )
print(modelRotRate)

In [ ]:
#It takes a 0.055 degree azimuth offset (~ 200 arcsec) to agree with the actualPositions.
modelRotRate = (
    -EARTH_ROTATION
    * np.cos(SIMONYI_LOCATION.lat.rad)
    * np.cos((expRecord.azimuth - 0.055) * u.deg)
    / np.cos((90.0 - expRecord.zenith_angle) * u.deg)
    / 3600.0
)
print(modelRotRate)

In [ ]:
print(az, (expRecord.azimuth) * u.deg)
print(el, (90.0 - expRecord.zenith_angle))

In [ ]:
expId = 2025091100243
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)

(mountErrors, mountData) = calculateMountErrors(expRecord, client)
#print(mountErrors)

names = ['X', 'Y', 'Z', 'U', 'V']
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.8, hspace=0.6)
plt.suptitle(f"Hexapod errors {expId}", fontsize=18, y=1.0)
scale = [(-100 , 100), (-100, 100), (-100, 100), (-10, 10), (-10, 10)]
for i in range(5):
    times = mountData.camhexData['private_efdStamp']
    times -= times[0]
    cam = mountData.camhexData[f'error{i}']
    #dem = mountData.camhexData[f'demand{i}']
    if i < 3:
        axs[0][i].set_ylabel("Microns")
    else:
        cam *= 3600
        dem *= 3600
        axs[0][i].set_ylabel("Arcseconds")
    axs[0][i].scatter(times, cam, marker='x', label='Measured')
    #axs[0][i].scatter(times, dem, marker='o', label='Demand')
    axs[0][i].set_title(f'Cam{names[i]}')
    axs[0][i].set_ylim(scale[i][0], scale[i][1])
for i in range(5):
    times = mountData.m2hexData['private_efdStamp']
    times -= times[0]
    m2 = mountData.m2hexData[f'error{i}']
    #dem = mountData.m2hexData[f'demand{i}']
    if i < 3:
        axs[1][i].set_ylabel("Microns")
    else:
        m2 *= 3600
        dem *= 3600
        axs[1][i].set_ylabel("Arcseconds")
    axs[1][i].scatter(times, m2, marker='x', label='Measured')
    #axs[1][i].scatter(times, dem, marker='o', label='Demand')
    axs[1][i].set_title(f'M2{names[i]}')
    axs[1][i].set_ylim(scale[i][0], scale[i][1])
    #if i == 4:
        #axs[1][i].legend(loc='center right')
plt.savefig(f"/home/c/cslage/u/Hexapods/data/Hexapod_Errors_{expId}.png")


In [ ]:
times  = np.array([2460971.50788252, 2460971.5078922,  2460971.50790188, 2460971.50791156
, 2460971.50792124, 2460971.50793091, 2460971.50794059, 2460971.50795027
, 2460971.50795995, 2460971.50796963, 2460971.50797931, 2460971.50798899
, 2460971.50799867, 2460971.50800835, 2460971.50801803, 2460971.50802771
, 2460971.50803739, 2460971.50804706, 2460971.50805674, 2460971.50806642
, 2460971.5080761,  2460971.50808578, 2460971.50809546, 2460971.50810514
, 2460971.50811482, 2460971.5081245,  2460971.50813418, 2460971.50814386
, 2460971.50815354, 2460971.50816321, 2460971.50817289, 2460971.50818257
, 2460971.50819225, 2460971.50820193, 2460971.50821161, 2460971.50822129
, 2460971.50823097, 2460971.50824065])
times *= 86400.0
times -= times[0]

azs = [-32.64328583, -32.65446805, -32.66564802, -32.67682575, -32.68800124
, -32.69917449, -32.71034549, -32.72151424, -32.73268076, -32.74384503
, -32.75500706, -32.76616685, -32.77732439, -32.78847969, -32.79963275
, -32.81078356, -32.82193213, -32.83307846, -32.84422255, -32.8553644
, -32.866504,  -32.87764136, -32.88877648, -32.89990935, -32.91103999
, -32.92216838, -32.93329453, -32.94441844, -32.9555401,  -32.96665953
, -32.97777671, -32.98889165, -33.00000435, -33.01111481, -33.02222302
, -33.03332899, -33.04443273, -33.05553422]

In [ ]:
plt.plot(times, azs, marker='x')

In [ ]:
plt.plot(azs, marker='x')

In [ ]:
print(len(mountData.azimuthData), len(mountData.elevationData))

In [ ]:
azModelValues, elModelValues = getAltAzOverPeriod(expRecord, nPoints=len(mountData.azimuthData))
azimuthData = mountData.azimuthData
azValues = np.asarray(azimuthData["actualPosition"])
azMedian = np.median(azValues)
azModelMedian = np.median(azModelValues)
# subtract the overall offset                                                                                                     
azModelValues -= azModelMedian - azMedian

In [ ]:
plt.plot(azModelValues, marker='x')

In [ ]:
azModelValues, _ = getAltAzOverPeriod(expRecord, nPoints=len(mountData.azimuthData))
_, elModelValues = getAltAzOverPeriod(expRecord, nPoints=len(mountData.elevationData))

azimuthData = mountData.azimuthData
azValues = np.asarray(azimuthData["actualPosition"])
azMedian = np.median(azValues)
azModelMedian = np.median(azModelValues)
# subtract the overall offset                                                                                                     
azModelValues -= azModelMedian - azMedian
plt.plot(azModelValues, marker='x')

In [ ]:
plt.plot(azModelValues, marker='x')

In [ ]:
plt.plot(azimuthData.linearModel.values, marker='x')
plt.plot(azValues, marker='o')
plt.plot(azimuthData.demandPosition.values, marker='+')

In [ ]:
azimuthData['linearModel'].plot()
azimuthData['actualPosition'].plot()
azimuthData['demandPosition'].plot(ls='--')

In [ ]:
newMountData =  getAzElRotHexDataForExposure(client, expRecord)

In [ ]:
newMountData.azimuthData['actualPosition'].plot()
newMountData.azimuthData['demandPosition'].plot()


In [ ]:
plt.plot(newMountData.azimuthData.actualPosition.values, marker='+')

In [ ]:
newMountData.azimuthData.columns

In [ ]:
times = mountData.azimuthData.actualPositionTimestamp.values
times -= times[0]
azs = mountData.azimuthData.actualPosition.values

In [ ]:
plt.plot(times, azs, marker='x')

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
plt.suptitle(f"Time delay between TMA data points")
for j, expId in enumerate([2025102200082, 2025102300193]):
    dataId = {'exposure':expId, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    (mountErrors, mountData) = calculateMountErrors(expRecord, client)
    times = mountData.azimuthData.actualPositionTimestamp.values
    times -= times[0]
    azs = mountData.azimuthData.actualPosition.values
    deltas = []
    for i, time in enumerate(times):
        if i == 0:
            continue
        delta = times[i] - times[i-1]
        deltas.append(delta)
    axs[j].set_title(f"{expId}\n Median delay = {np.median(deltas):.3f} seconds")
    axs[j].hist(deltas, bins=20, range=(0,5))
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/EFD_Data_Delays.png")
    

In [ ]:
plt.hist(deltas, bins=10, range=(0,5))